### Word embedding for wine reviews and sentiment analysis

In [ ]:
# development method either "local" or "remote"
development = "local"

In [ ]:
# root path if local
root_path = "."

if development == "remote":
    
    from google.colab import drive

    # mounting google drive to system
    drive.mount('/content/drive')
    
    # root path if remote
    root_path = '/content/drive/My Drive/word_embedding'

# module path
module_path = root_path + "/..";

# model path
model_path = root_path + "/model"

In [ ]:
import sys
    
sys.path.insert(0, module_path)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import helper as hlp
import nltk
import re

from abc import ABC, abstractmethod
from string import punctuation

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score, mean_squared_error

from sklearn.ensemble import RandomForestRegressor as RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor as DecisionTreeRegressor
from nltk.corpus import stopwords
from collections import Counter

from torch.utils.data import TensorDataset, DataLoader

from pandas.api.types import is_categorical_dtype
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
# loading data
raw = pd.read_csv(root_path + "/data/wines/wine_reviews.csv", low_memory = False);

# dropping unnecessary column
raw = raw.drop(columns = ["Unnamed: 0"], inplace = False)

In [ ]:
def transform_text(data, column, punctuation):
    ''' utility function for text transformation for the machine to interpret '''
    
    # make dataframe copy
    data_copy = data.copy()
    
    # transform each row to lowercase
    data_copy[column] = data_copy[column].str.lower()
    
    # filter out punctuation
    data_copy[column] = data_copy[column].str.replace('[^\w\s]', '')
        
    return data_copy

In [ ]:
# transform each review to lowercase and remove punctuation
raw_data = transform_text(raw, "description", punctuation)

In [ ]:
# transform non-numerical data to categorical
hlp.trans_categorical(raw_data, labels = ["description"])

# transform/normalize numeric data
raw_numeric_data = hlp.transform_to_numeric(raw_data, suffle_data_frame = True)

# training and validation set
train_set, valid_set = hlp.split_data(raw_numeric_data, threshold = 1 / 8)

# targets; scores for each product and features
train_dataset, valid_dataset = hlp.split_features(train_set, valid_set, columns = ["description", "points"])

In [ ]:
transformer = TfidfVectorizer(stop_words = stopwords.words('english'))

In [ ]:
# learning vocabulary and get the term-document matrix
train_term_document = transformer.fit_transform(train_dataset[1])

# based on learned vocabulary transform validation set
valid_term_document = transformer.transform(valid_dataset[1])

In [ ]:
train_term_document

### Logistic Regression

In [ ]:
# create our model
model = LogisticRegression()

# fit data
model.fit(train_term_document, train_dataset[2])

In [ ]:
print(r2_score(valid_dataset[2], model.predict(valid_term_document)))

In [ ]:
def validation_verbose(description, model, transformer, expected_score = None):
    
    # generate tf-idf-weighted document-term matrix
    description_tdm = transformer.transform([description])

    # predict score given description
    score = model.predict(description_tdm)[0]

    print(f"{description:100.100}...\
          \n\t output {score}, expected {expected_score}\n")

In [ ]:
def validation(data, model, transformer, count = 5):
    
    for index in range(count):
        
        # current description
        description = data[0]["description"][index]
        
        # expected output
        score = validation_set[1][index]
        
        validation_verbose(description, model, transformer, expected_score = score)

In [ ]:
# some input from validation set
validation(validation_set, lr, tfidf, count = 5)

In [ ]:
# some custom input

# average score
validation_verbose("Pretty bad, can't handle the taste, extremely sour, how can someone make such wine?", lr, tfidf)

# good score
validation_verbose("Amazing, fine vintage, delicious, rich texture that sobbing for more takes, just pure quality.", lr, tfidf)

### Word Embedding Algorithms:
    
1. Embedding Layer
2. Word2Vec
    1. CBOW
    2. C. Skip-Gram
3. GloVe

In [ ]:
# batch size
batch_size = 16

# training and validation set
train_set, valid_set = hlp.split_data(raw_data, threshold = 1 / 8, batch_trim = batch_size)

# targets; scores for each product and features
train_dataset, valid_dataset = hlp.split_features(train_set, valid_set, columns = ["description", "points"])

In [ ]:
class Chainer(ABC):
    ''' chainer class for chaining text transformations '''
    
    @abstractmethod
    def process(self, data, chain = None):
        ''' chain method for data preprocessing '''
        
        pass
    
class Tokenize(Chainer):
    
    def process(self, data, chain):
        ''' chain method for data preprocessing '''
        
        # tokenize, split a sentence by space
        chain.data = data.str.split()
        
        # find maximum size of sequence of tokens
        chain.sequence_max = max([ len(sequence) for sequence in chain.data ])
        
        return (chain.data, chain)
    
class Vocabulary(Chainer):
    
    def process(self, data, chain):
        ''' create the known vocabulary basis '''
        
        # count token occurrences
        chain.tokens = Counter([ token for sequence in data for token in sequence ])
        
        # vocabulary_size
        chain.vocabulary_size = len(chain.tokens) + 1
        
        # word to integer mapping, 0 is reserved for padding
        chain.word_to_int = { key : (index + 1) for index, key in enumerate(chain.tokens) }
        
        # integer to word mapping
        chain.int_to_word = { index : word for word, index in chain.word_to_int.items() }
        
        return (data, chain)
    
class NumericToToken(Chainer):
    
    def process(self, data, chain):
        ''' apply textual transformation '''
        
        assert(hasattr(chain, 'int_to_word'))
        
        # transform from textual to numerical representation
        chain.data = [ 
            [ chain.int_to_word[token] for token in sequence if token in chain.int_to_word ] 
        for sequence in data ]
        
        return (chain.data, chain)

class TokenToNumeric(Chainer):
    
    def process(self, data, chain):
        ''' apply numerical transformation '''

        assert(hasattr(chain, 'word_to_int'))

        # transform from textual to numerical representation
        chain.data = [ 
            [ chain.word_to_int[token] if token in chain.word_to_int else 0 for token in sequence ] 
        for sequence in data ]

        return (chain.data, chain)
           
class Filler(Chainer):
    
    def process(self, data, chain):
        ''' apply padding to numerical content '''

        # assert numerical representation of input data
        assert(all(isinstance(token, int) for sequence in data for token in sequence))
        assert(hasattr(chain, 'sequence_max'))

        # get the real size of each sequence
        chain.sizes = np.array([ len(sequence) - 1 for sequence in data ])

        # transform by padding
        chain.data = [ sequence + [0] * (chain.sequence_max - len(sequence)) for sequence in data ]
        
        return (np.array(chain.data), chain)

class Composer(Chainer):
    
    def __init__(self, transforms):
        
        self.transforms = transforms
    
    def process(self, data, chain = None):
        
        # initialize chainer data
        self.data = data
        
        # available chainer
        chain = self if chain is None else chain
        
        # apply transformations in series
        for transform in self.transforms:
            
            # check if it's chainer transformer
            if(isinstance(transform, Chainer)):
                
                # update existing data, pass only current class reference
                self.data, _ = transform.process(self.data, chain)
                
            else:
                
                # list item is not an instance of Chainer transformer
                raise Exception("Illegal parameter, provide contiguous set of Chainer(s)")
                
        return (self.data, self)
    
    def transform(self, data, chain):
        
        # apply transformation given learned vocabulary
        features, _ = Composer([
            Tokenize(),
            TokenToNumeric(),
            Filler()
        ]).process(data, chain = chain)
        
        return (features, chain)

In [ ]:
# apply transformations and learn the vocabulary of our train dataset
train_dataset[1], train_chainer = Composer([
    Tokenize(),
    Vocabulary(),
    TokenToNumeric(),
    Filler()
]).process(train_dataset[1])


# apply transformations given existing learned vocabulary
valid_dataset[1], valid_chainer = chainer.transform(valid_dataset[1], chainer)

In [ ]:
# create the tensor datasets
train_tensor_dataset = TensorDataset(torch.from_numpy(train_dataset[1]).long(), torch.from_numpy(train_dataset[2].values), torch.from_numpy(chainer.sizes))
valid_tensor_dataset = TensorDataset(torch.from_numpy(valid_dataset[1]).long(), torch.from_numpy(valid_dataset[2].values), torch.from_numpy(chainer.sizes))

# create data loaders
train_loader = DataLoader(train_tensor_dataset, batch_size = batch_size, shuffle = True)
valid_loader = DataLoader(valid_tensor_dataset, batch_size = batch_size, shuffle = True)

### Embedding Layer

In [ ]:
class Model(nn.Module):
    
    def __init__(self, num_embeddings, embedding_dim, hidden_size, num_layers, output_size, bidirectional = False):
        super(Model, self).__init__()
        
        self.num_embeddings = num_embeddings
        self.embedding_dim = embedding_dim
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size
        self.bidirectional = bidirectional
        
        # sparse(embedding) layer
        self.embedding = nn.Embedding(self.num_embeddings, self.embedding_dim)
        
        # recurrent neural network layer(lstm)
        self.rnn = nn.LSTM(self.embedding_dim, self.hidden_size, self.num_layers, 
                               bidirectional = self.bidirectional, batch_first = True)
        
        # fully connected layer(linear) + dropout
        self.sec1 = nn.Sequential(
            nn.Linear(self.hidden_size, 128),
            nn.Dropout(0.1)
        )
        
        
        # fully connected layer(linear) + dropout
        self.sec2 = nn.Sequential(
            nn.Linear(128, self.output_size),
            nn.Dropout(0.1)
        )
    
    def forward(self, x, indices, verbose = False):

        # embed words into dense representation
        x = self.embedding(x)
        
        # recurrent neural network; pass forward
        x, _ = self.rnn(x)
        
        # stack rnn output
        x = x.contiguous()
        
        # fully connected layer; pass forward while casually dropping cells
        x = self.sec1(x)
        
        # fully connected layer; pass forward while casually dropping cells
        x = self.sec2(x)
        
        # extract last prediction
        x = x.view(batch_size, -1)
        
        return x[torch.arange(len(indices)), indices]
    
    def save_model(self, checkpoint):
        ''' save model to dictionary '''
        
        assert(isinstance(checkpoint, dict))
        
        checkpoint["num_embeddings"] = self.num_embeddings
        checkpoint["embedding_dim"] = self.embedding_dim
        checkpoint["hidden_size"] = self.hidden_size
        checkpoint["num_layers"] = self.num_layers
        checkpoint["output_size"] = self.output_size
        checkpoint["bidirectional"] = self.bidirectional
        
        # save model internal weights
        checkpoint["state_dict"] = self.state_dict()
        
        return checkpoint
        
    @staticmethod
    def load_model(checkpoint):
        ''' load model from dictionary '''
        
        assert(isinstance(checkpoint, dict))
        
        # create an instance of Model
        model = Model(
            checkpoint["num_embeddings"],
            checkpoint["embedding_dim"],
            checkpoint["hidden_size"],
            checkpoint["num_layers"],
            checkpoint["output_size"],
            checkpoint["bidirectional"]
        )
        
        # load weights
        model.load_state_dict(checkpoint["state_dict"])
        
        return model

### Defining hyperparameters

In [ ]:
class Context():
    
    def __init__(self, model, learning_rate, verbose = True):
        
        assert(isinstance(model, Model))
        
        # device to be used
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        if verbose:
            
            print(self.device)
        
        # model to be trained
        self.model = model.to(self.device)
        
        # learning rate
        self.learning_rate = learning_rate

        # loss function (mean squared error loss)
        self.criterion = nn.MSELoss(reduction = 'mean')

        # optimizer with momentum
        self.optimizer = optim.Adam(self.model.parameters(), lr = self.learning_rate)
        
        # current validation min
        self.valid_loss_min = np.inf
    
    def create_scheduler(self):
        
        # custom scheduler
        self.scheduler = optim.lr_scheduler.MultiStepLR(self.optimizer, milestones = [4, 9], gamma = 0.72)
        
    def save_context(self, name):
        
        # current checkpoint to be saved
        checkpoint = {
            "state_optim" : self.optimizer.state_dict(),
            "learning_rate" : self.learning_rate,
            "valid_loss_min" : self.valid_loss_min
        }
        
        # save model params to dict
        checkpoint = self.model.save_model(checkpoint)
        
        # save checkpoint to disk
        torch.save(checkpoint, f"{model_path}/{name}")
        
    @staticmethod
    def load_context(name, context = None):
        
        # load latest checkpoint
        checkpoint = torch.load(f"{model_path}/{name}")
        
        # model loading
        model = Model.load_model(checkpoint)
        
        # recreate context
        context = Context(model, checkpoint["learning_rate"])
        
        # update current validation loss
        context.valid_loss_min = checkpoint["valid_loss_min"]
        
        # optimizer loading e.g state momentum
        context.optimizer.load_state_dict(checkpoint["state_optim"])
        
        return context

In [ ]:
# our model
model = Model(chainer.vocabulary_size, 512, 256, 2, 1)

# context container for model characteristics
context = Context(model, learning_rate = 0.0001)

### Model testing

In [ ]:
# singleton batch sample
feature_sample, target_sample, indices = next(iter(train_loader))

# move to corresponding available
feature_sample, target_sample = feature_sample.to(context.device), target_sample.to(context.device)

# forward pass
output = model.forward(feature_sample, indices)

# compute lossop
loss = context.criterion(output, target_sample)

print(f"Loss is {loss.item()}")

In [ ]:
def valid_model(context, epoch, train_acc_loss):
    ''' model validation '''
    
    # the model to be validated
    model = context.model
        
    # validation loss
    valid_acc_loss = 0
    
    # evaluation mode
    model.eval()

    with torch.no_grad():

        for features, targets, indices in valid_loader:

            # forward pass
            output = model.forward(features, indices)

            # calculate loss
            loss = context.criterion(output, targets)
            
            # update validation acc loss
            valid_acc_loss += loss.item() 

    # average loss 
    valid_acc_loss = valid_acc_loss / len(valid_loader)

    # print training / validation statistics 
    print('Current Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch + 1, train_acc_loss, valid_acc_loss))

    if(valid_acc_loss <= context.valid_loss_min):
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            context.valid_loss_min, valid_acc_loss))

        # update current best validation loss
        context.valid_loss_min = valid_acc_loss

        # save best model
        context.save_context('model.pt')
        

def train_model(context, epochs = 1, show_every_step = len(train_loader)):
    ''' model training '''
    
    # the model to be trained
    model = context.model
    
    # train loss and step counter
    train_acc_loss, step_counter = 0, 0
    
    for epoch in range(epochs):
        
        # training mode
        model.train()
        
        for index, (features, targets, indices) in enumerate(train_loader):
            
            # removing accumulated gradients
            context.optimizer.zero_grad()
            
            # move to corresponding available device
            features, targets = features.to(context.device), targets.to(context.device)
            
            # forward pass
            output = model.forward(features, indices)

            # calculate loss
            loss = context.criterion(output, targets)
            
            # calculate gradients
            loss.backward()
            
            # adjusting weights
            context.optimizer.step()
            
            # update loss acc
            train_acc_loss += loss.item()
            
            # update step counter
            step_counter += 1
            
            if(step_counter % show_every_step == 0):
                
                # average loss 
                train_acc_loss = train_acc_loss / len(train_loader)

                # print training / validation statistics 
                print('Current Epoch: {} \t Training Loss: {:.6f}'.format(
                    epoch + 1, train_acc_loss))
                
                # reset train and step accumulator
                train_acc_loss, step_counter = 0, 0

In [ ]:
# training model
train_model(context, 5, show_every_step = 1500)

### Testing model

In [ ]:
def pred_model(context):
    
    # evaluation mode
    context.model.eval()
    
    # valid features targets 
    features, targets, indices = next(iter(train_loader))
    
    # move to available device
    features, targets = features.to(context.device), targets.to(context.device)
    
    # making prediction
    outputs = context.model.forward(features, indices, verbose = True)
    
    for i in range(len(features)):
        
        print(f"Predicted score: {outputs[i]}, actual score {targets[i]}")

In [ ]:
# making predictions
pred_model(context)